<h1><span style="color:red">Named Entity Recognition for SuAVE</span></h1>

This notebook uses spaCy to generate named entity tags by parsing a selected text field in a SuAVE dataset
See https://spacy.io/ for more information.

The following tags are generated and added as #multi variables to survey datafile (from https://spacy.io/api/annotation#named-entities), if they exist in text:

   * PERSON:	People, including fictional.
   * NORP:	Nationalities or religious or political groups.
   * FAC:	Buildings, airports, highways, bridges, etc.
   * ORG:	Companies, agencies, institutions, etc.
   * GPE:	Countries, cities, states.
   * LOC:	Non-GPE locations, mountain ranges, bodies of water.
   * PRODUCT:	Objects, vehicles, foods, etc. (Not services.)
   * EVENT:	Named hurricanes, battles, wars, sports events, etc.
   * WORK_OF_ART:	Titles of books, songs, etc.
   * LAW:	Named documents made into laws.
   * LANGUAGE:	Any named language.
   * DATE:	Absolute or relative dates or periods.


Additionally, users have an option to add user-defined dictionaries of terms, and add custom #multi variables with terms from the dictionary. These cells are optional. Users can also load larger pre-trained NER models.



For testing:

http://localhost:8888/notebooks/Downloads/jupyter-suave/operations/tagger/NER.ipynb?surveyurl=http://suave-dev.sdsc.edu/main/file=spatialsuave_Russian_FB_Ads_w_Concepts.csv&views=1110001&view=grid&user=spatialsuave&csv=spatialsuave_Russian_FB_Ads_w_Concepts.csv&params=none&dzc=https://maxim.ucsd.edu/dzgen/lib-staging-uploads/bea6f8abb86c98ef168775a159612828/content.dzc&activeobject=null

## 1. Retrieve survey parameters from the URL

In [8]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("survey_url='".concat(getQueryStringValue("surveyurl")).concat("'"));
IPython.notebook.kernel.execute("views='".concat(getQueryStringValue("views")).concat("'"));
IPython.notebook.kernel.execute("view='".concat(getQueryStringValue("view")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
IPython.notebook.kernel.execute("csv_file='".concat(getQueryStringValue("csv")).concat("'")); 
IPython.notebook.kernel.execute("dzc_file='".concat(getQueryStringValue("dzc")).concat("'")); 
IPython.notebook.kernel.execute("params='".concat(getQueryStringValue("params")).concat("'")); 
IPython.notebook.kernel.execute("active_object='".concat(getQueryStringValue("activeobject")).concat("'")); 
IPython.notebook.kernel.execute("full_notebook_url='" + window.location + "'"); 

<IPython.core.display.Javascript object>

## 2. Setting up the environment (if needed) and importing libraries

<h2><span style="color:red">Skip this cell if the spaCy enviroment is already set up. Otherwise, un-comment and run the following commands to set up the environment.  </span></h2>

In [9]:
#### Install the main module (see https://spacy.io/)
# ! pip install spacy

#### lemmatization - only needed if creating a model from scratch
# !pip install -U spacy-lookups-data

####  Need to install one of these models
# !python -m spacy download en_core_web_lg   # 789 mb
# !python -m spacy download en_core_web_md   # 91 mb
# !python -m spacy download en_core_web_sm   # 11 mb

#### Installing these models via pip (see https://pypi.org/project/spacy/)
    
# !pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.5/en_core_web_sm-2.2.5.tar.gz    
# !pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_md-2.2.5/en_core_web_md-2.2.5.tar.gz    
# !pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.5/en_core_web_lg-2.2.5.tar.gz    



### Import spacy and other libraries, load the default pre-trained spacy model (small)

In [17]:
# common imports
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Markdown, display

import pandas as pd
pd.set_option('display.max_colwidth', 0)
    
import numpy as np
import panel as pn

pn.extension("tabulator")
def printmd(string):
    display(Markdown(string))

absolutePath = "/home/jovyan/temp_csvs/"

# local imports
import sys
sys.path.insert(1, '../../helpers')
import panel_libs as panellibs
import suave_integration as suaveint

# specific imports
import requests
import re


In [11]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

In [12]:
import spacy

# Currently installed is en_core_web_sm model version 2.2.5. It's size is 11 mb
# To update the small model (en_core_web_sm), uncomment and run 
# !python -m spacy download en_core_web_sm

# Install and load the small model:

!python -m spacy download en_core_web_sm

import en_core_web_sm
nlp = en_core_web_sm.load()

<h2><span style="color:red">Optionally, uncomment the lines below to install and import larger pretrained NER models. Otherwise, skip to step 3</span></h2>

In [13]:
#### Installing medium or large models will take a bit longer:

#### For the medium model:
# !python -m spacy download en_core_web_md   # 91 mb
#### and load it using
# import en_core_web_md
# nlp = en_core_web_md.load()

#### Or, for the large model:
# !python -m spacy download en_core_web_lg   # 789 mb
#### and load it using
# import en_core_web_lg
# nlp = en_core_web_lg.load()



## 3. Select a survey file from SuAVE or import a local CSV file

In [14]:
data_select = pn.widgets.RadioBoxGroup(name='Select notebook', options=['Load survey file from SuAVE', 
                                                                        'Import a local CSV file'], 
                                       inline=False)
data_select

RadioBoxGroup(name='Select notebook', options=['Load survey file from Su...], value='Load survey f...)

In [18]:
data_input = pn.widgets.FileInput()
    
def check_selection():
    if data_select.value == 'Load survey file from SuAVE':
        global fname
        fname = absolutePath + csv_file
        printmd("<b><span style='color:red; font-size: 200%;'>Current SuAVE survey will be loaded. Continue to step 4.</span></b>")

    else:
        message = pn.pane.HTML("<b><span style='color:red; font-size: 200%;'>Upload data and continue to step 4.</span><br><span style='font-size: 150%;'>IMPORTANT: The local CSV file should not have SuAVE-specific variable names!</span></b>", width=700)
        return pn.Column(message, data_input)
    
check_selection()

Column
    [0] HTML(str, width=700)
    [1] FileInput()

## 4. Visualize the data and select a text variable to parse

In [ ]:
if not pd.isnull(data_input.filename):
    fname = absolutePath + data_input.filename
    data_input.save(fname)

df = panellibs.extract_data(fname)
panellibs.slider(df)


In [ ]:
# if the above doesn't work:
with pd.option_context("display.max_columns", None):
    if any("geometry" in col for col in df.columns):
        display(df.drop(['geometry'],axis=1))
    else:
        display(df)
 

<h2>4a. Run this cell to define fields to be added, and generate respective #multi variables.</h2>

Then skip to Step 5 unless you want to add a user-defined disctionary in Step 4b

In [ ]:
ent_labels = ['PERSON', 'NORP', 'FAC', 'ORG', 'GPE', 'LOC','PRODUCT', 'EVENT', 'WORK_OF_ART','LAW','LANGUAGE', 'DATE']
col_labels = ['nerPerson#multi', 'nerPopulation Group#multi', 'nerFacility#multi', 'nerOrganization#multi', 'nerAdministrative Area#multi', 'nerLocation#multi','nerProduct#multi', 'nerEvent#multi', 'nerWork of Art#multi','nerLegal Document#multi','nerLanguage#multi', 'nerDate#multi']

varcols = df.columns.tolist()
# remove any variable names are unlikely to contain parsable text 
varcols = [x for x in varcols if '#number' not in x and '#date' not in x and '#img' not in x and '#href' not in x and '#link' not in x]

# Left panel
left_text = pn.Row("####Select Variables for NER", margin=(0,0,-15,270))
binary_selector = pn.widgets.CrossSelector(options=varcols, width=630)
left_panel = pn.Column(left_text, binary_selector, css_classes=['widget-box'], margin=(0,30,0,0))

remap_text = pn.pane.Markdown('###      Make selections and run section 5, or optionally define additional entities to extract in section 4b', width=650)

# Display widgets
widgets = pn.Row(left_panel)
full_display = pn.Column(widgets,remap_text)
full_display

<h2><span style="color:red">4b. Optionally: add a user defined dictionary to the pipeline, and use entity matcher to generate an additional #multi variable</span></h2>
Alternatively, skip to step 5

In [ ]:
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from spacy.language import Language
class EntityMatcher(object):
    name = "entity_matcher"

    def __init__(self, nlp, terms, label):
        patterns = [nlp.make_doc(text) for text in terms]
        self.matcher = PhraseMatcher(nlp.vocab)
        self.matcher.add(label, None, *patterns)

    def __call__(self, doc):
        matches = self.matcher(doc)
        for match_id, start, end in matches:
            span = Span(doc, start, end, label=match_id)
            doc.ents = list(doc.ents) + [span]
        return doc

@Language.component("entity_matcher")
def entity_matcher(doc):
    return doc

In [ ]:
# Example of adding a #multi variable and a corresponding vocabulary to extract

added_column = "nerAnimal#multi"
added_group = "ANIMAL"
terms = ("cat", "dog", "tree kangaroo", "giant sea spider", "monkey")

# adding the new #multi variable to the list of entities to extract

entity_matcher = EntityMatcher(nlp, terms, added_group)
nlp.add_pipe("entity_matcher", after="ner")
ent_labels.append(added_group)
col_labels.append(added_column)




## 5. Generate pre-defined #multi variables by doing NER over the selected text variable

## 6. Run spaCy

In [ ]:
# ent_labels = entity_selector.value

print(binary_selector.value)

def properize(txt):
    if len(txt) > 3:
        txt = txt.title()
    return txt
def extract_entity(doc, label):
    return '|'.join(list(set([properize(ent.text) for ent in doc.ents if ent.label_ == label])))
def extract_all(doc):
    data = {}
    for col_label, ent_label in zip(col_labels, ent_labels):
        data[col_label] = extract_entity(doc, ent_label)
    return pd.Series(data)

# Replace NA with empty in each row
# Convert row to string
# Join row with spaces
concatted = df[binary_selector.value].fillna('').astype(str).dropna().apply(lambda row: ' '.join(row), axis=1)

# Apply nlp and then extract
# extracted_df = concatted.head().apply(nlp).apply(extract_all)
extracted_df = concatted.progress_apply(nlp).apply(extract_all)

df_new = pd.concat([df, extracted_df], axis=1)
print('Dimensions:\n --- The original df: ' +str(df.shape) +'\n --- The ner-generated df: '+ str(extracted_df.shape)+'\n --- The concatenated df:' +str(df_new.shape))


## 7. Only keep columns with more than N records as facets (for new fields, or for all fields), make others #hidden

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# set min number of values in a column
mincount = 0

# whether to drop columns from the original dataframe as well
drop_from_original=False

if drop_from_original:
    start_column = 0
else:
    start_column = len(df.columns)

for column in df_new.columns:
    if (df_new.columns.get_loc(column) > start_column) & ((np.int16(df_new[column].count()).item() - len(df_new[df_new[column] == ''])) < mincount):
        if column.find("#multi") > 0:
            new_col = column.split('#')[0]+"#multi#hidden"
        elif column.find("#link") > 0:
            new_col = column
        else:
            new_col = column.split('#')[0]+"#hidden"

        df_new = df_new.rename(columns={column:new_col})
        print(column + " renamed to "+ new_col)
            


## 8. Examine the generated dataframe

In [ ]:
panellibs.slider(df_new)

In [ ]:
# if the above doesn't work:
with pd.option_context("display.max_columns", None):
    if any("geometry" in col for col in df_new.columns):
        display(df_new.drop(['geometry'],axis=1))
    else:
        display(df_new)
 

In [ ]:
# now write this back, or upload to SuAVE.

updated_df = df_new.copy().fillna('')


## 9. Generate a new survey and open it in SuAVE

In [ ]:
if data_select.value == 'Import a local CSV file':
    csv_file = data_input.filename
    dzc_file = ''
    
new_file = suaveint.save_csv_file(updated_df, absolutePath, csv_file)

In [ ]:
#Input survey name
import ipywidgets as widgets
input_text = widgets.Text(placeholder='Enter Survey Name...')
output_text = widgets.Text()

def bind_input_to_output(sender):
    output_text.value = input_text.value

# Tell the text input widget to call bind_input_to_output() on submit
input_text.on_submit(bind_input_to_output)

printmd("<b><span style='color:red'>Input survey name here, press Enter, and then run the next cell:</span></b>")
# Display input text box widget for input
display(input_text)

display(output_text)


In [ ]:
#Print survey name
survey_name = output_text.value
printmd("<b><span style='color:red'>Survey Name is: </span></b>" + survey_name)

In [ ]:
suaveint.create_survey(survey_url,new_file, survey_name, dzc_file, user, csv_file, view, views, data_select.value)